In [189]:
import numpy as np
import helpers 

datapath = "./dataset/"


x_train, x_test, y_train, train_ids, test_ids = helpers.load_csv_data(datapath)

In [190]:
data_path = "./dataset/x_train.csv"
feature_names =  np.genfromtxt(data_path, max_rows=2, delimiter=',', names=True).dtype.names
columns = np.asarray(feature_names[0:])

In [191]:
#for i in range(len(columns)):
#    print("{} - {}: {}".format(i, columns[i], len(np.array([x for x in x_train[:,i] if np.isnan(x)]))))

Vous trouvez pas que la partie Record identification ne sert à rien ? En vrai on peut garder la variable _STATE mais FMONTH je trouve que ça sert à rien

Maybe do the one hot encoding for _STATE 

I think we should not consider IDATE, we should directly consider IMONTH, IDAY, IYEAR since it corresponds to the same thing, we should consider the columns to potentially drop

In [192]:
x_new = np.copy(x_train) 
arrays_to_drop = []
arrays_to_drop.append("IDATE")

What should we do with DISPCODE ? Maybe remove all interviews that are incomplete, but this depends maybe the other infos are crucial even if it not unfinished, I think by standerizing it will be 0 or 1, nothing to do now

I think we should drop the seqno, since it is not a relevant info, as well as _PSU, CTELENUM

In [193]:
arrays_to_drop.append("SEQNO")
arrays_to_drop.append("_PSU")
arrays_to_drop.append("CTELENUM")

What to do with PVTRESD1, COLGHOUS, STATERES, CELLFON3 ?

LADULT has a lot of missing values (pretty much only NAN) we are going to drop this column anyway 

Process for PHYSHLTH, MENTHLTH, POORHLTH 
The values corresponds to days, between 1-30 it is already good.
We're going to replace all 88 values by 0 because it corresponds to None which is 0 days, and we will assume that those who refused or did not answer have median values.

In [194]:
def process_HLTH(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] == 88: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 30]        
    median = np.nanmedian(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column


In [195]:

indice = [i for i, item in enumerate(feature_names) if item == "PHYSHLTH"][0]
x_new[:, indice] = process_HLTH(x_train[:, indice])

In [196]:
variables_for_process_HLTH = ["PHYSHLTH","MENTHLTH", "POORHLTH"]
for col in variables_for_process_HLTH :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1][0]
    x_new[:,indice] = process_HLTH(x_train[:,indice])

Process for values that have as answer Yes or No
We're going to replace No by 0 and yes by 1, since yes is already by 1, we're going to change 2 by 0. For values 7 which corresponds to don't know or not sure, 9 for refused, and BLANK for missing values, we're going to take the median

We're going to use this method for variables : HLTHPLN1, MEDCOST, BPMEDS, BLOODCHO, TOLDHI2, CVDINFR4, CVDCRHD4, CVDSTRK3, ASTHMA3, ASTHNOW, CHCSCNCR, CHCOCNCR, CHCCOPD1, HAVARTH3, ADDEPEV2, CHCKIDNY, 

In [197]:
def process_2(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] == 2: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 1]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7) or (new_column[i] == 9) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

Je trouve pas feature name CVDINFR4, CVDCRHD4 dans le dataset mais je le trouve dans le document

In [198]:
variables_for_process2 = ["HLTHPLN1", "MEDCOST", "BPMEDS", "BLOODCHO", "TOLDHI2",
                          "CVDSTRK3", "ASTHMA3", "ASTHNOW",
                          "CHCSCNCR", "CHCOCNCR", "CHCCOPD1", "HAVARTH3", "ADDEPEV2",
                          "CHCKIDNY", "CPDEMO1", "VETERAN3", "INTERNET", "PREGNANT",
                         "QLACTLM2", "USEEQUIP", "BLIND", "DECIDE", "DIFFWALK", "DIFFDRES",
                          "DIFFALON", "SMOKE100","STOPSMK2", "EXERANY2",
                         "LMTJOIN3", "ARTHDIS2", "FLUSHOT6", "PNEUVAC3",
                         "HIVTST6", "PDIABTST", "INSULIN", "DIABEYE", "DIABEDU"]
for col in variables_for_process2 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_2(x_train[:,indice])

Process for CHECKUP1, we have to check what to do with never have been checked I gave 0 but I think we should instead do 5
see how we can do both CHOLCHK and CHECKUP1

In [199]:
def process_3(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] == 8: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 4]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7) or (new_column[i] == 9) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [200]:
variables_for_process3 = ["CHECKUP1"]
for col in variables_for_process3 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_3(x_train[:,indice])

Have to see for BPHIGH4, DIABETE3, PREDIAB1
Ask question for was this only when you were pregnant

See process for DIABAGE2, how old were you (ages between 1-97) ?

In [201]:
def process_4(column):
    new_column = column

    filtered_elements = [x for x in new_column if 1 <= x <= 97]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 98) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [202]:
variables_for_process4 = ["DIABAGE2"]
for col in variables_for_process4 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_4(x_train[:,indice])

Tu m'as dit t'as fait one hot encoding for SEX : male, female

See process for CHILDREN, values between 1-87, None corresponds to 88 we will transform to 0

In [203]:
def process_5(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] == 88: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 87]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 98) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [204]:
variables_for_process5 = ["CHILDREN"]
for col in variables_for_process5 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_5(x_train[:,indice])

On discute de comment faire pour INCOME2

See process for WEIGHT2, For weight we will put everything in kilograms, so for values between 50-0999 which is in pound we will transform it it kg, and for values between 9000-9998 we will remove the first column of the value because it only corresponds to the fact that the value is in kg.

See process for HEIGHT3, we will transform everything in centimeters, from inches to centimers

In [205]:
def process_6(column, conversion):
    new_column = column
    
    for i in range(len(new_column)):
        if new_column[i] >= 50 and new_column[i] <= 999: 
            new_column[i] = new_column[i] * conversion
        elif new_column[i] >= 9000 and new_column[i] <= 9998: 
            new_column[i] = new_column[i] - 9000
            
    filtered_elements = [x for x in new_column if 0 <= x <= 999]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7777) or (new_column[i] == 9999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [206]:
variables_for_process6 = ["WEIGHT2", "HEIGHT3"]
pounds_to_kg = 0.453592
inch_to_cm = 2.54
conversion = [pounds_to_kg, inch_to_cm]

for i, col in enumerate(variables_for_process6) :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_6(x_train[:,indice], conversion[i])

We do process for ALCDAY5 where :
- the value in the form 1_ _ = days per week, there are 4 week per month
- 2_ _ = days in the past 30 day
- 888 no drinks we will then consider 0
- for the values 777, 999, BLANK, we will consider the mean

The new value corresponds to the number of days per month you consumed at least one drink

Same thing works for EXEROFT1

In [223]:
def process_7(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] >= 101 and new_column[i] <= 199: 
            new_column[i] = (new_column[i] - 100) * 4
        elif new_column[i] >= 201 and new_column[i] <= 299: 
            new_column[i] = (new_column[i] - 200)
        elif (new_column[i] == 888): 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 99]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 777) or (new_column[i] == 999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [226]:
variables_for_process7 = ["ALCDAY5", "EXEROFT1", "EXEROFT2", "STRENGTH"]
for col in variables_for_process7 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_7(x_train[:,indice])

Do process for SMOKDAY2, LASTSMK2, can't find STOPSMK2

Do process for Alcohol, especially for AVEDRNK2 and the other, we are considering if it is BLANK because it is not asked since the previous answer is no (888), which is the answer to the question have you had any alcoholic beverage. Since the answer to the variable name ALCDAY5 is no they will not ask the other questions. 

In [219]:
def process_8(column, column_ALCDAY5):
    new_column = column
    for i in range(len(new_column)):
        if (np.isnan(new_column[i]) and column_ALCDAY5[i] == 888): 
            new_column[i] = 0

            
    filtered_elements = [x for x in new_column if 0 <= x <= 76]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [220]:
variables_for_process8 = ["AVEDRNK2", "DRNK3GE5", "MAXDRNKS" ]
indice_ALCDAY5 = [i for i, item in enumerate(feature_names) if item.find("ALCDAY5") != -1 ][0]
for col in variables_for_process8 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_8(x_train[:,indice], x_train[:,indice_ALCDAY5])

Process for FRUITJU1, FRUIT1, FVBEANS, FVGREEN, FVORANG, VEGETAB1  . We are going to use  information about FRUITJUI1 to understand how the values in FRUIT1 works, indeed,
- the value in the form 1_ _ = times per day, we will consider there are 30 days per month
- 2_ _ = times per week, we will consider there are 4 weeks per month (maybe you want to consider more 4,..)
- 300 means less than one time per month, we will consider 0 
- 3_ _ = times per month
- 555 never we will then consider 0
- for the values 777, 999, BLANK we will consider the median

In [221]:
 def process_9(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] >= 101 and new_column[i] <= 199: 
            new_column[i] = (new_column[i] - 100) * 30
        elif new_column[i] >= 201 and new_column[i] <= 299: 
            new_column[i] = (new_column[i] - 200) * 4
        elif (new_column[i] == 300) or (new_column[i] == 555): 
            new_column[i] = 0
        elif new_column[i] >= 301 and new_column[i] <= 399: 
            new_column[i] = (new_column[i] - 300)

            
    filtered_elements = [x for x in new_column if 0 <= x <= 99]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 777) or (new_column[i] == 999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [222]:
variables_for_process9 = ["FRUITJU1", "FRUIT1", "FVBEANS",
                          "FVGREEN", "FVORANG", "VEGETAB1"]
for col in variables_for_process9 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_9(x_train[:,indice])

Maybe do one hot encoding + consider the other values for EXRACT11,EXRACT21 categorie values

We do process for EXERHMM1, we have values in hours and minutes, we want it to be continious, we have _ _ _, the first value correspond to the number of hours, the two other values correspond to minutes, We will convert everything to minutes 
Also works for EXERHMM2

In [230]:
 def process_10(column):
    new_column = column
    for i in range(len(new_column)):
        if ((new_column[i] >= 1 and new_column[i] <= 759) 
        or (new_column[i] >= 800 and new_column[i] <= 959)):             
            first_digit = new_column[i] // 100
            digits2_3 = new_column[i] % 100           
            new_column[i] = first_digit * 60 + digits2_3

            
    filtered_elements = [x for x in new_column if 0 <= x <= 600]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 777) or (new_column[i] == 999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [231]:
variables_for_process10 = ["EXERHMM1", "EXERHMM2"]
for col in variables_for_process10 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_10(x_train[:,indice])

Do for ARTHSOCL, SEATBELT

Do JOINPAIN

DO FLSHTMY2, HIVTSTD3
You have value Month/Year

DO IMFVPLAC

We will process the values for BLDSUGAR, FEETCHK2
We want to have the times they checked something per year:
- the value in the form 1_ _ = times per day, we will consider there are 365 days in a year
- 2_ _ = times per week, we will consider there are 52 weeks per year (you may want to consider there are more (52,143))
- 3_ _ = times per month, we consider there are 12 months in a year
- 4_ _ = times per year
- 555 no feet, we will then consider 0
- 888 never, whichi means 0
- for the values 777, 999, BLANK we will consider the median


In [232]:
def process_11(column):
    
    new_column = column
    
    for i in range(len(new_column)):
        if new_column[i] >= 101 and new_column[i] <= 199: 
            new_column[i] = (new_column[i] - 100) * 365
        elif new_column[i] >= 201 and new_column[i] <= 299: 
            new_column[i] = (new_column[i] - 200) * 52
        elif new_column[i] >= 301 and new_column[i] <= 399: 
            new_column[i] = (new_column[i] - 300) * 12
        elif new_column[i] >= 401 and new_column[i] <= 499: 
            new_column[i] = new_column[i] - 400
        elif (new_column[i] == 300) or (new_column[i] == 555): 
            new_column[i] = 0


            
    filtered_elements = [x for x in new_column if 0 <= x <= 99]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 777) or (new_column[i] == 999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [233]:
variables_for_process11 = ["BLDSUGAR", "FEETCHK2"]
for col in variables_for_process11 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_11(x_train[:,indice])

Process for DOCTDIAB, CHKHEMO3, FEETCHK
We want all the values to be between 0-76 that corresponds to how many times they did something
- 88 which corresponds to None/ 0
- 98 never heard of means they have done it 0 times
- for the values 77-99-BLANK we will again consider the median

In [234]:
def process_12(column):
    new_column = column
    for i in range(len(new_column)):
        if (new_column[i] == 88 or new_column[i] == 98 ): 
            new_column[i] = 0

    filtered_elements = [x for x in new_column if 0 <= x <= 76]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [235]:
variables_for_process12 = ["BLDSUGAR", "FEETCHK2"]
for col in variables_for_process12 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new[:,indice] = process_12(x_train[:,indice])

DO EYEEXAM SIMILAR TO PREVIOUS THAT I HAVEN'T DONE

In [236]:
arrays_to_drop.append("PAINACT2")
arrays_to_drop.append("QLMENTL2")

same for a lot of them we will remove them same as Feature_Engineering